In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

%cd /content/drive/MyDrive/kaggle

/content/drive/MyDrive/kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c veri-bilimi-ve-yapay-zeka-okulu-kaggle-yarismasi

  0% 0.00/38.5k [00:00<?, ?B/s]
100% 38.5k/38.5k [00:00<00:00, 1.35MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  veri-bilimi-ve-yapay-zeka-okulu-kaggle-yarismasi.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
data_dir = "/content/drive/MyDrive/kaggle"

data_df = pd.read_csv("sample_submission.csv")
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               310 non-null    int64  
 1   value_increased  0 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 5.0 KB


In [ ]:
train_df.head()

train_df.info()

train_df.isnull().mean()

print(train_df.isnull().sum())

print(train_df.isnull().sum()/ len(train_df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 56 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      465 non-null    int64  
 1   value_increased         465 non-null    bool   
 2   Ability                 465 non-null    float64
 3   Potential               43 non-null     float64
 4   Positions               465 non-null    object 
 5   Caps / Goals            16 non-null     object 
 6   Foot                    465 non-null    object 
 7   Height                  465 non-null    object 
 8   Weight                  465 non-null    object 
 9   Aerial Reach            32 non-null     float64
 10  Command of Area         32 non-null     float64
 11  Communication           32 non-null     float64
 12  Eccentricity            32 non-null     float64
 13  First Touch             465 non-null    float64
 14  Handling                32 non-null     fl

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')


train_data['Height'] = train_data['Height'].str.replace(' CM', '').astype(float)
train_data['Weight'] = train_data['Weight'].str.replace(' KG', '').astype(float)

test_data['Height'] = test_data['Height'].str.replace(' CM', '').astype(float)
test_data['Weight'] = test_data['Weight'].str.replace(' KG', '').astype(float)

outfield_players = train_data[train_data['Positions'] != 'GK'].copy()

threshold = 0.8
outfield_players = outfield_players.dropna(thresh=int((1 - threshold) * len(outfield_players)), axis=1)

for column in outfield_players.columns:
    if outfield_players[column].isnull().any():
        if outfield_players[column].dtype == 'object':
            outfield_players[column].fillna(outfield_players[column].mode()[0], inplace=True)
        else:
            outfield_players[column].fillna(outfield_players[column].median(), inplace=True)


outfield_players['BMI'] = outfield_players['Weight'] / ((outfield_players['Height'] / 100) ** 2)
test_data['BMI'] = test_data['Weight'] / ((test_data['Height'] / 100) ** 2)


position_mapping = {
    'LW': 'Forward', 'RW': 'Forward', 'ST': 'Forward',
    'CAM': 'Midfielder', 'CM': 'Midfielder', 'CDM': 'Midfielder',
    'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender',
}
outfield_players['Positions'] = outfield_players['Positions'].replace(position_mapping)
test_data['Positions'] = test_data['Positions'].replace(position_mapping)


categorical_features = ['Positions', 'Foot']
numerical_features = ['Height', 'Weight', 'BMI']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

X_train_outfield = outfield_players.drop(columns=['value_increased', 'id'])
y_train_outfield = outfield_players['value_increased']


X_train_outfield_processed = preprocessor.fit_transform(X_train_outfield)


smote = SMOTE(random_state=42)
X_train_outfield_balanced, y_train_outfield_balanced = smote.fit_resample(X_train_outfield_processed, y_train_outfield)

def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = X_train_outfield_balanced.shape[1]
model = build_model(input_shape)

model.fit(X_train_outfield_balanced, y_train_outfield_balanced, epochs=60, batch_size=8, validation_split=0.2)

X_test_outfield = test_data[test_data['Positions'] != 'GK'].drop(columns=['id'])
X_test_outfield_processed = preprocessor.transform(X_test_outfield)

test_predictions_outfield = model.predict(X_test_outfield_processed)
test_predictions_outfield = (test_predictions_outfield > 0.5).astype(bool)


X_test_goalkeepers = test_data[test_data['Positions'] == 'GK'].drop(columns=['id'])


test_data['value_increased'] = None
test_data.loc[test_data['Positions'] != 'GK', 'value_increased'] = test_predictions_outfield.flatten()
test_data.loc[test_data['Positions'] == 'GK', 'value_increased'] = False


submission = test_data[['id', 'value_increased']]
submission.to_csv('sample_submission.csv', index=False)


y_train_pred = model.predict(X_train_outfield_balanced)
y_train_pred = (y_train_pred > 0.5).astype(bool)

print("Classification Report:")
print(classification_report(y_train_outfield_balanced, y_train_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_train_outfield_balanced, y_train_pred))

y_train_proba = model.predict(X_train_outfield_balanced)
print("ROC-AUC:", roc_auc_score(y_train_outfield_balanced, y_train_proba))

print("Test set class distribution (after predictions):")
print(test_data['value_increased'].value_counts())


Epoch 1/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.4974 - loss: 0.7630 - val_accuracy: 0.0000e+00 - val_loss: 6.0383
Epoch 2/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5260 - loss: 0.7187 - val_accuracy: 0.0000e+00 - val_loss: 2.4678
Epoch 3/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6318 - loss: 0.6747 - val_accuracy: 0.0000e+00 - val_loss: 1.2066
Epoch 4/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6023 - loss: 0.6599 - val_accuracy: 0.4422 - val_loss: 0.7438
Epoch 5/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6224 - loss: 0.6692 - val_accuracy: 0.3333 - val_loss: 0.8208
Epoch 6/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6325 - loss: 0.6368 - val_accuracy: 0.3129 - val_loss: 0.8141
Epoch 7/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6383 - loss: 0.6371 - val_accuracy: 0.2653 - val_loss: 0.8700
Epoch 8/60
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6557 - loss: 0.6302 - val_accuracy: 0.340